In [1]:
import os
import bz2
import numpy as np
import matplotlib.pyplot as plt
import netCDF4
from calendar import monthrange
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
mpl.style.use('../stylefiles/socib.mplstyle')
%matplotlib inline

In [2]:
sstmin, sstmax = 19., 29.
lonmin, lonmax, latmin, latmax = -7.0, 36.0, 30.0, 47.0
odyssea_dir = "/data_local/Satellite/SST_Odyssea"
figdir = "/data_local/Satellite/figures/SST_comparison"

In [3]:
m = Basemap(projection='merc', llcrnrlon=lonmin, llcrnrlat=latmin + 0.1,
                    urcrnrlon=lonmax,urcrnrlat=latmax,  
                    lat_ts=0.5 * (latmin + latmax),
                    resolution='c')

In [4]:
def extract_bz2(datafile, datafileout):
    with open(datafile_ODYSSEA, "rb") as f_in:
        with open(datafileout, "wb") as f_out:
            f_out.write(bz2.decompress(f_in.read()))

In [5]:
def load_sst(datafile):
    with netCDF4.Dataset(datafile) as nc:
        sst = nc.variables['analysed_sst'][:].squeeze()
        lon = nc.variables['lon'][:]
        lat = nc.variables['lat'][:]
    return lon, lat, sst

In [6]:
def plot_sst_comparison(lonMUR, latMUR, sstMUR, 
                        lonODYSSEA, latODYSSEA, sstODYSSEA,
                        lonOISST, latOISST, sstOISST, 
                        figtitle, figname, **kwargs):
    
    sstmin, sstmax = 19.5, 28.5
    fig = plt.figure(figsize=(8, 12))
    plt.suptitle(figtitle, fontsize=22)
    
    plt.subplot(311)
    plt.title('MUR', fontsize=18)
    pcm = m.pcolormesh(lonMUR, latMUR, sstMUR - 273.15, 
                       vmin=sstmin, vmax=sstmax, cmap=plt.cm.RdYlBu_r)
    plt.subplot(312)
    plt.title('ODYSSEA', fontsize=18)
    m.pcolormesh(lonODYSSEA, latODYSSEA, sstODYSSEA - 273.15, 
                 vmin=sstmin, vmax=sstmax, cmap=plt.cm.RdYlBu_r)
    plt.subplot(313)
    plt.title('MyOcean', fontsize=18)
    m.pcolormesh(lonOISST, latOISST, sstOISST - 273.15, 
                 vmin=sstmin, vmax=sstmax, cmap=plt.cm.RdYlBu_r)

    fig.subplots_adjust(right=0.85)
    cbar_ax = fig.add_axes([0.875, 0.15, 0.05, 0.7])
    cbar = fig.colorbar(pcm, cax=cbar_ax, cmap=plt.cm.RdYlBu_r, extend='both')
    cbar.set_label('$^{\circ}$C', rotation=0, ha='left')
    plt.savefig(figname, dpi=300)
    # plt.show()
    plt.close()

In [7]:
# Loop on time
yearstart, yearend = 2016, 2016
monthstart, monthend = 8, 8
daystart, dayend = 1, 11

for yy in range(yearstart, yearend + 1):
    year = str(yy)
    for mm in range(monthstart, monthend + 1):
        month = str(mm).zfill(2)
 
        if mm == monthstart:
            daystart0 = daystart
            dayend0 = monthrange(yy, mm)[1]
        elif mm == monthend:
            daystart0 = 1
            dayend0 = dayend
        else:
            daystart0 = daystart
            dayend0 = dayend
                
        for dd in range(daystart0, dayend0 + 1):
            day = str(dd).zfill(2)
            
            datafile_MUR = ("/data/current/opendap/observational/satellite/ocean_temperature/jpl/sst/L4"
                            "/{0}/{1}/{0}{1}{2}-JPL-L4UHfnd-GLOB-v01-fv04-MUR.nc".format(year, month, day))
            datafile_ODYSSEA = ("/data/current/opendap/observational/satellite/ocean_temperature/ifremer/sst/L4"
                                "/{0}/{1}/{0}{1}{2}-IFR-L4_GHRSST-SSTfnd-ODYSSEA-"
                                "MED_002-v2.0-fv1.0.nc.bz2".format(year, month, day))
            datafile_OISST = ("/data/current/opendap/observational/satellite/ocean_temperature/myocean/sst/L4"
                              "/{0}/{1}/{0}{1}{2}000000-GOS-L4_GHRSST-SSTfnd-"
                              "OISST_HR_NRT-MED-v02.0-fv02.0.nc".format(year, month, day))

            datafile_ODYSSEA_out = os.path.join(odyssea_dir, os.path.basename(datafile_ODYSSEA).split('.')[0] + '.nc')
            extract_bz2(datafile_ODYSSEA, datafile_ODYSSEA_out)
            datafile_ODYSSEA_out
            
            lonMUR, latMUR, sstMUR = load_sst(datafile_MUR)
            lonOISST, latOISST, sstOISST = load_sst(datafile_OISST)
            lonODYSSEA, latODYSSEA, sstODYSSEA = load_sst(datafile_ODYSSEA_out)
            
            
            llonMUR, llatMUR = np.meshgrid(lonMUR, latMUR)
            lonMUR, latMUR = m(llonMUR, llatMUR)
            llonOISST, llatOISST = np.meshgrid(lonOISST, latOISST)
            lonOISST, latOISST = m(llonOISST, llatOISST)
            llonODYSSEA, llatODYSSEA = np.meshgrid(lonODYSSEA, latODYSSEA)
            lonODYSSEA, latODYSSEA = m(llonODYSSEA, llatODYSSEA)
            
            figtitle = '{0}-{1}-{2}'.format(year, month, day)
            figname = os.path.join(figdir, 'sst_comparison_{0}_{1}_{2}'.format(year, month, day))
            
            plot_sst_comparison(lonMUR, latMUR, sstMUR, 
                        lonODYSSEA, latODYSSEA, sstODYSSEA,
                        lonOISST, latOISST, sstOISST, 
                        figtitle, figname, vmin=sstmin, vmax=sstmax)

RuntimeError: No such file or directory

'2016_07_30'